# # CitiBike Predictive Model

In this code, I use the prepped dataframe from prior steps that merged the weather data with the citibike data. I run a series of tests on sample data to determine the right variables to use in my Linear Regression model. To start, I imported all necessary packages, including the model packages from SciKitLearn

Step 1) I cleaned the data to improve my accuracy results. Any data that would be considered outliers would skew the accuracy of my results.

Step 2) To Prepare to dummy my categorical variables, i created new columns in the dataframe for day of week and month of year

Step 3) I took a random sample (6.7%) of the master dataframe to make the data more manageable when running tests

Step 4) Using this random sample, I dummified all my categorical data such as gender, usertype, weather event, day of week and month of year

Step 5) I concatenated these dummies with the original random sample

Step 6) I created a coefficient matrix and indexed only the tripduration column to determine which variables have the strongest impact on tripduration. I will be initially using these variables in the master model

Step 7) Normalize the continuous variables that will impact trip duration because the scales of those variable are all different but because those scales are different it will impact the accuracy of the result

Step 8) User the original master dataframe to create dummies of the same one i used in the random sample and then concanate this dataframe with the original 

Step 9) Take a random sample (8.5%) of this new concatenated master dataframe to have one more manageable to work with 

Step 10) Create two dataframes, one with only tripduration (the target variable) and one with all the remaining predictor variables

Step 11) Create your test and training sets, with 20% for testing and 80% for training.

Step 12) Run the model and calculate the R^2 to reveal model accuracy

In [1]:
#import pandas as pd
import pandas as pd

In [2]:
#import numpy as np
import numpy as np

In [3]:
#import datetime as dt
import datetime as dt

In [4]:
#import the Linear Regression package from SK Learn to create the linear regression model
from sklearn.linear_model import LinearRegression

#import the train,test split package to succeessfully create a model that uses a proper testing and training set 
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

In [6]:
ls

2017 Citibike Data.csv              201710-citibike-tripdata.csv
201701-citibike-tripdata.csv        201711-citibike-tripdata.csv
201702-citibike-tripdata.csv        201712-citibike-tripdata.csv
201703-citibike-tripdata.csv        CitiBike Data for Machine.csv
201704-citibike-tripdata.csv        CitiBike Prediction Notebook.ipynb
201705-citibike-tripdata.csv        CitiBike_Data_for_Machine1.csv
201706-citibike-tripdata.csv        Citibike Data Analysis.ipynb
201707-citibike-tripdata.csv        Untitled.ipynb
201708-citibike-tripdata.csv        Wunderground Weather Data.csv
201709-citibike-tripdata.csv        attribute_histogram_plots.png


In [7]:
#import the prepped dataframe that contains weather data and start date and age and distance
master_df=pd.read_csv("CitiBike_Data_for_Machine1.csv")
master_df.head()

/Users/Matthew.Giancristofaro@ibm.com/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,index,tripduration,starttime,stoptime,start station name,start station latitude,start station longitude,end station id,end station name,...,Sea Level Press. (in) low,Visibility (mi) high,Visibility (mi) avg,Visibility (mi) low,Wind (mph) high,Wind (mph) avg,Wind (mph) high.1,Precip. (in),Event,day of week
0,0,0,11.333333,2017-01-01 00:00:21,2017-01-01 00:11:41,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,...,29.89,10,10,10,12,5,23,0.0,0,6
1,1,1,21.366667,2017-01-01 00:00:45,2017-01-01 00:22:08,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,...,29.89,10,10,10,12,5,23,0.0,0,6
2,2,2,10.800000,2017-01-01 00:00:57,2017-01-01 00:11:46,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,...,29.89,10,10,10,12,5,23,0.0,0,6
3,3,3,10.516667,2017-01-01 00:01:10,2017-01-01 00:11:42,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,...,29.89,10,10,10,12,5,23,0.0,0,6
4,4,4,10.350000,2017-01-01 00:01:25,2017-01-01 00:11:47,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,...,29.89,10,10,10,12,5,23,0.0,0,6


In [8]:
#clean the data

#drop all rows where age > 70
master_df=master_df[master_df["age"]<70]

#drop all rows where trip distance is 0
master_df=master_df[master_df["Distance in km"] != 0]

#drop all rows where trip duration is > 2.5 hours
master_df=master_df[master_df["tripduration"]<60]

In [9]:
round(master_df.describe(),2)

,Unnamed: 0,index,tripduration,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,...,Sea Level Press. (in) avg,Sea Level Press. (in) low,Visibility (mi) high,Visibility (mi) avg,Visibility (mi) low,Wind (mph) high,Wind (mph) avg,Wind (mph) high.1,Precip. (in),day of week
count,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,14349740.00,...,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00,15816543.00
mean,8187934.11,755327.01,13.04,40.74,-73.98,1340.05,40.74,-73.98,23768.42,1979.19,...,30.02,29.93,9.97,9.12,6.92,12.87,4.77,20.88,0.07,2.85
std,4734311.91,493664.56,9.36,0.07,0.12,1341.78,0.04,0.06,5346.14,11.35,...,0.20,0.22,0.23,1.60,3.58,3.44,2.04,5.97,0.25,1.93
min,0.00,0.00,1.02,0.00,-74.03,72.00,0.00,-74.07,14529.00,1948.00,...,29.38,28.92,7.00,3.00,0.00,7.00,1.00,7.00,0.00,0.00
25%,4081448.50,340592.50,6.18,40.72,-74.00,359.00,40.72,-74.00,18665.00,1971.00,...,29.91,29.82,10.00,9.00,3.00,10.00,3.00,17.00,0.00,1.00
50%,8193120.00,681345.00,10.22,40.74,-73.99,485.00,40.74,-73.99,25477.00,1982.00,...,30.01,29.93,10.00,10.00,9.00,13.00,5.00,20.00,0.00,3.00
75%,12296319.50,1141285.00,17.47,40.76,-73.98,3140.00,40.76,-73.98,28122.00,1988.00,...,30.14,30.04,10.00,10.00,10.00,15.00,6.00,24.00,0.01,4.00
max,16364656.00,1897591.00,59.98,40.81,0.00,3654.00,40.81,0.00,33481.00,2001.00,...,30.68,30.61,10.00,10.00,10.00,33.00,18.00,48.00,3.03,6.00


In [ ]:
list(master_df.columns.values)

In [10]:
#convert the day of week column to strings, prepping to be dummified
master_df['day of week']=master_df['day of week'].astype(str)

In [11]:
#create a new column called month of year, and convert the type of data to string variables to prep for dummies
master_df['Month of Year']=pd.to_datetime(master_df['startday']).dt.month.astype(str)

In [12]:
#create a dataframe of a random samplesize of the master dataframe to determine which variables impact the target variable most
master_df_rand = master_df.sample(frac = 0.067, random_state = 1)
master_df_rand = master_df_rand.reset_index(drop=True)
master_df_rand

,Unnamed: 0,index,tripduration,starttime,stoptime,start station name,start station latitude,start station longitude,end station id,end station name,...,Visibility (mi) high,Visibility (mi) avg,Visibility (mi) low,Wind (mph) high,Wind (mph) avg,Wind (mph) high.1,Precip. (in),Event,day of week,Month of Year
0,4355490,794098,17.816667,2017-05-17 08:37:25,2017-05-17 08:55:15,E 6 St & Avenue B,40.724537,-73.981854,264,Maiden Ln & Pearl St,...,10,10,10,12,6,23,0.00,0,2,5
1,8125000,1308746,7.133333,2017-07-25 09:24:48,2017-07-25 09:31:56,Broadway & W 51 St,40.762288,-73.983362,359,E 47 St & Park Ave,...,10,10,9,10,5,14,0.00,0,1,7
2,6164666,1080006,7.483333,2017-06-20 14:19:25,2017-06-20 14:26:55,E 20 St & Park Ave,40.738274,-73.987520,474,5 Ave & E 29 St,...,10,8,3,15,5,24,0.03,Rain,1,6
3,1941131,422808,5.666667,2017-03-21 09:37:53,2017-03-21 09:43:33,W 20 St & 11 Ave,40.746745,-74.007756,3224,W 13 St & Hudson St,...,10,10,10,13,4,18,0.00,0,1,3
4,7893811,1077557,25.050000,2017-07-20 18:03:05,2017-07-20 18:28:09,Grand Army Plaza & Central Park S,40.764397,-73.973715,3316,W 104 St & Amsterdam Ave,...,10,9,4,12,3,17,0.01,Rain,3,7
5,3707978,146586,8.233333,2017-05-03 15:00:18,2017-05-03 15:08:32,Jay St & Tech Pl,40.695065,-73.987167,430,York St & Jay St,...,10,10,10,18,8,31,0.00,0,2,5
6,14160481,16440,7.683333,2017-11-01 09:44:48,2017-11-01 09:52:30,Greenpoint Ave & Manhattan Ave,40.730260,-73.953940,3119,Vernon Blvd & 50 Ave,...,10,10,10,10,3,16,0.00,0,2,11
7,10215072,1663219,12.800000,2017-08-29 08:54:54,2017-08-29 09:07:43,Barrow St & Hudson St,40.731724,-74.006744,327,Vesey Pl & River Terrace,...,10,9,4,15,8,21,0.10,Rain,1,8
8,16042482,567792,8.666667,2017-12-17 09:43:25,2017-12-17 09:52:05,Congress St & Clinton St,40.688153,-73.995209,2000,Front St & Washington St,...,10,10,10,9,3,12,0.00,0,6,12
9,15895945,421255,7.933333,2017-12-12 06:04:43,2017-12-12 06:12:39,Cleveland Pl & Spring St,40.722104,-73.997249,491,E 24 St & Park Ave S,...,10,10,6,17,7,33,0.00,Snow,1,12


In [13]:
list(master_df_rand.columns.values)

['Unnamed: 0',
 'index',
 'tripduration',
 'starttime',
 'stoptime',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender',
 'Distance in km',
 'startday',
 'age',
 'speed in m/s',
 'Date',
 'Temp high (F)',
 'Temp avg (F)',
 'Temp low (F)',
 'Dew Point high',
 'Dew Point avg',
 'Dew Point low',
 'Humidity high',
 'Humidity avg',
 'Humidity low',
 'Sea Level Press. (in) high',
 'Sea Level Press. (in) avg',
 'Sea Level Press. (in) low',
 'Visibility (mi) high',
 'Visibility (mi) avg',
 'Visibility (mi) low',
 'Wind (mph) high',
 'Wind (mph) avg',
 'Wind (mph) high.1',
 'Precip. (in)',
 'Event',
 'day of week',
 'Month of Year']

In [14]:
#create dummy columns of the categorical variables including usertype, gender, event, day of week and month of year
df1_dummies = pd.get_dummies(master_df_rand[[
                                     'usertype',
                                     'gender',
                                     'Event','day of week','Month of Year']])

In [15]:
#check to ensure all the dummies were created successfully
df1_dummies.shape

(1059708, 28)

In [16]:
#take a dataframe of all the important variables needed in the dataframe
df1=master_df_rand[['tripduration',
 'Distance in km',
 'age',
 'Temp high (F)',
 'Temp avg (F)',
 'Temp low (F)',
 'Dew Point high',
 'Dew Point avg',
 'Dew Point low',
 'Humidity avg',
 'Humidity low',
 'Visibility (mi) avg',
 'Visibility (mi) low',
 'Wind (mph) high',
 'Wind (mph) avg',
 'Precip. (in)']]




In [17]:
#concatanate the predictor variables with the dummies dataframe
dftotal=pd.concat([df1,df1_dummies],axis=1)
dftotal.head()

,tripduration,Distance in km,age,Temp high (F),Temp avg (F),Temp low (F),Dew Point high,Dew Point avg,Dew Point low,Humidity avg,...,Month of Year_11,Month of Year_12,Month of Year_2,Month of Year_3,Month of Year_4,Month of Year_5,Month of Year_6,Month of Year_7,Month of Year_8,Month of Year_9
0,17.816667,2.895861,29.0,90,78,65,63,56,46,47,...,0,0,0,0,0,1,0,0,0,0
1,7.133333,1.066154,34.0,71,67,62,61,59,57,79,...,0,0,0,0,0,0,0,1,0,0
2,7.483333,0.768949,54.0,83,77,71,70,62,51,64,...,0,0,0,0,0,0,1,0,0,0
3,5.666667,0.784819,42.0,59,51,42,31,25,21,40,...,0,0,0,1,0,0,0,0,0,0
4,25.050000,3.899637,27.0,94,86,77,72,68,62,58,...,0,0,0,0,0,0,0,1,0,0


In [18]:
#determine which factors have the gratest impact on trip duration by creating a coefficient matrix and calling the
#tripduration column and put it in descending order
corr_matrix = dftotal.corr()
corr_matrix["tripduration"].sort_values(ascending=False)

tripduration           1.000000
usertype_Customer      0.301741
gender_Unknown         0.255058
Distance in km         0.121349
Temp high (F)          0.106860
Temp avg (F)           0.103846
Temp low (F)           0.097826
Dew Point avg          0.082207
Dew Point low          0.080687
Dew Point high         0.080153
day of week_6          0.060216
day of week_5          0.057814
Event_0                0.036148
gender_Female          0.034641
Month of Year_7        0.032917
Month of Year_8        0.030886
Month of Year_6        0.023799
Month of Year_9        0.020737
Visibility (mi) avg    0.019302
Month of Year_4        0.017264
Month of Year_5        0.014908
Visibility (mi) low    0.010110
Month of Year_10       0.001792
Humidity avg          -0.010453
day of week_4         -0.015445
Event_Rain            -0.018287
day of week_0         -0.020290
day of week_3         -0.021543
Precip. (in)          -0.021591
Event_0               -0.022293
Humidity low          -0.022581
day of w

In [19]:
list(dftotal.columns.values)

['tripduration',
 'Distance in km',
 'age',
 'Temp high (F)',
 'Temp avg (F)',
 'Temp low (F)',
 'Dew Point high',
 'Dew Point avg',
 'Dew Point low',
 'Humidity avg',
 'Humidity low',
 'Visibility (mi) avg',
 'Visibility (mi) low',
 'Wind (mph) high',
 'Wind (mph) avg',
 'Precip. (in)',
 'usertype_Customer',
 'usertype_Subscriber',
 'gender_Female',
 'gender_Male',
 'gender_Unknown',
 'Event_0',
 'Event_0',
 'Event_Rain',
 'Event_Snow',
 'day of week_0',
 'day of week_1',
 'day of week_2',
 'day of week_3',
 'day of week_4',
 'day of week_5',
 'day of week_6',
 'Month of Year_1',
 'Month of Year_10',
 'Month of Year_11',
 'Month of Year_12',
 'Month of Year_2',
 'Month of Year_3',
 'Month of Year_4',
 'Month of Year_5',
 'Month of Year_6',
 'Month of Year_7',
 'Month of Year_8',
 'Month of Year_9']

In [20]:
#normalize the continuous variables that will impact tripduration
master_df['Distance in km']=std.fit_transform(master_df[['Distance in km']])
master_df['age']=std.fit_transform(master_df[['age']])
master_df['Temp high (F)']=std.fit_transform(master_df[['Temp high (F)']])

In [21]:
###Work with the master dataframe and get dummies of the categorical variables
#the start and end stations names are not included in the dummified data set because my computer could not process 
#that many variables with that many rows. The process was taking 3+ hours and the kernel had failed. Therefore,
#this model is under the assumption that the 'distance in km' column considers all possible combinations of 
#start and end stations travelled. Therefore the model will still work with just the distance in km column. In theory,
#if a user inputted the start and end station, the machine would determine the distance between the two points,
#which is why the distance in km column accomodates for that.
master_dummies=pd.get_dummies(master_df[[
                                     'usertype',
                                     'gender',
                                     'Event','day of week','Month of Year']])

In [22]:
#create a new master dataframe by concatanating the original master and dummies dataframe
master_df_total=pd.concat([master_df,master_dummies],axis=1)

In [23]:
#create a dataframe with a random sample of the dataframe and run a test linear regression to determine accuracy
df_rs = master_df_total.sample(frac = 0.085, random_state = 1)

In [24]:
list(df_rs.columns.values)

['Unnamed: 0',
 'index',
 'tripduration',
 'starttime',
 'stoptime',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender',
 'Distance in km',
 'startday',
 'age',
 'speed in m/s',
 'Date',
 'Temp high (F)',
 'Temp avg (F)',
 'Temp low (F)',
 'Dew Point high',
 'Dew Point avg',
 'Dew Point low',
 'Humidity high',
 'Humidity avg',
 'Humidity low',
 'Sea Level Press. (in) high',
 'Sea Level Press. (in) avg',
 'Sea Level Press. (in) low',
 'Visibility (mi) high',
 'Visibility (mi) avg',
 'Visibility (mi) low',
 'Wind (mph) high',
 'Wind (mph) avg',
 'Wind (mph) high.1',
 'Precip. (in)',
 'Event',
 'day of week',
 'Month of Year',
 'usertype_Customer',
 'usertype_Subscriber',
 'gender_Female',
 'gender_Male',
 'gender_Unknown',
 'Event_0',
 'Event_0',
 'Event_Rain',
 'Event_Snow',
 'day of week_0',
 'day of week_1',
 'day of wee

In [36]:
#create a dataframe of all the predictor variables that are important for the model based on the coefficient matrix

df_predictors=df_rs[[
 'Distance in km',
 'age',
 'Temp high (F)',
 'usertype_Subscriber','usertype_Customer',
 'gender_Male', 'gender_Female']]

In [37]:
#create dataframe with only the target variable, trip duration
df_tripduration=df_rs['tripduration']

In [38]:
#create the x,y training and testing variables from the dataframes. The size of the training set will be 80 percent
#the size of the testing set will be 20 percent the variables
x_train,x_test,y_train,y_test=train_test_split(df_predictors,df_tripduration,test_size=.2, random_state=1)

In [39]:
#simplify the linear regression function to lm
lm=LinearRegression()

In [40]:
#create the linear regression model using the lm.fit function on the x and y training sets
lm.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [41]:
#print the r^2 score of the linear regression model to determine the level of accuracy the contains
lm.score(x_test,y_test)

-77.85524220457263

In [43]:
#test the model using lm.predict and the x_test set of data
lm.predict(x_test)

array([15.75299596, 12.4528108 , 14.17259866, ...,  7.55394492,
       23.66334794, 16.30806628])

In [44]:
#import stats models to check the R^2 value of the model
import statsmodels.api as sm

In [45]:
r = sm.OLS(df_tripduration, df_predictors).fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           tripduration   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                 4.468e+05
Date:                Mon, 30 Apr 2018   Prob (F-statistic):               0.00
Time:                        17:24:19   Log-Likelihood:            -4.8330e+06
No. Observations:             1344406   AIC:                         9.666e+06
Df Residuals:                 1344399   BIC:                         9.666e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Distance in km          2.4733      0.015    163.245      0.000       2.444       2.503
age                     0.2761      0.011     25.934      0.000       0.255       0.297
Temp high (F)           0.6598      0.008     86.151      0.000       0.645       0.675
usertype_Subscriber    12.7933      0.048    266.120      0.000      12.699      12.888
usertype_Customer      21.8713      0.034    636.782      0.000      21.804      21.939
gender_Male            -1.1485      0.049    -23.221      0.000      -1.245      -1.052
gender_Female           0.5287      0.050     10.497      0.000       0.430       0.627
==============================================================================
Omnibus:                  1004455.528   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      14556141636.105
Skew:                          -1.746   Prob(JB):                         0.00
Kurtosis:                     512.745   Cond. No.                         14.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""